#  Training Sequential Circuits with NumPy
This notebook shows the expands upon the sequential circuits structure <br>
and optimize the circuit using NumPy.

In [1]:
#!pip install pennylane
#!pip install torch

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.embeddings import AmplitudeEmbedding
from model import Conv1DLayer, PoolingLayer, unitarity_conv1d, unitarity_pool, unitarity_toffoli
import torch

# Defining the Quantum device

In [3]:
#Specify how many wires will be used
num_wires = 12
num_ancilla = 4
num_work_wires = num_wires - num_ancilla

#Specify what type of device you are using.
#device_type = 'default.qubit'
#device_type = 'braket.local.qubit'
device_type = 'lightning.qubit'


#Initialize Device
dev = qml.device(device_type, wires=num_wires)

# Defining circuit to be trained

In [4]:
def circuit(num_wires, num_ancilla):
    @qml.qnode(dev, interface='autograd', diff_method ='adjoint')
    def func(inputs, params): 
        work_wires = list(range(num_wires - num_ancilla))
        ancilla_wires = list(range(len(work_wires),num_wires))

        # Data Embedding Layer
        AmplitudeEmbedding(inputs , wires=work_wires , normalize=True)

        # Hidden Layers
        work_wires , params = Conv1DLayer(unitarity_conv1d , 15)(work_wires , params)
        work_wires , params = PoolingLayer(unitarity_pool , 2)(work_wires , params)
        work_wires , params = Conv1DLayer(unitarity_conv1d , 15)(work_wires , params)
        work_wires , params = PoolingLayer(unitarity_pool , 2)(work_wires , params)

        #Toffili Structure
        unitarity_toffoli(work_wires,ancilla_wires)

        return [qml.expval(qml.PauliZ(wire)) for wire in ancilla_wires]
    return func

# Import training Data

In [5]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
batch_size = 32
embedding_size = 2**num_work_wires
labels = [0,1,2,3]  #,4,5,6,7]
num_labels = len(labels)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((16,16)),
    transforms.Lambda(lambda x: torch.reshape(x, (-1,))),
])

train_data = datasets.MNIST(
    root='./dataset/minst/',
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.MNIST(
    root='./dataset/minst/',
    train=False,
    download=True,
    transform=transform
)

#Filter for classes 
train_idx = [i for i, y in enumerate(train_data.targets) if y in labels]
train_data = torch.utils.data.Subset(train_data, train_idx)

test_idx = [i for i, y in enumerate(test_data.targets) if y in labels]
test_data = torch.utils.data.Subset(test_data, test_idx)

# Pytorch loader
train_loader = DataLoader(
    dataset=train_data,
    shuffle=True,
    batch_size=batch_size,
    drop_last = True
)

# Pytorch loader
test_loader = DataLoader(
    dataset=train_data,
    shuffle=False,
    batch_size=batch_size,
    drop_last = False
)

# Defining Training Functions

In [16]:
import autograd.numpy as anp 

def cross_entropy(targets, predictions):
    loss = 0
    for target, prediction in zip(targets, predictions):
        c_entropy = target * (anp.log(prediction[target])) + (1 - target) * anp.log(1 - prediction[1 - target])
        loss = loss + c_entropy
    return loss
    
def loss(params, data, targets):
    predictions = [circuit(num_wires, num_ancilla)(x,params) for x in data]
    return cross_entropy(labels, predictions)
        
    
def label(q_input):
    labels = []
    for val in q_input:
        l = np.argmax(val) 
        labels.append(l)
    return labels

def accuracy(predictions, labels):
    acc = 0
    predicted_labels = label(predictions)
    for l, p in zip(labels, predicted_labels):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)

# Defining Parameters and Hyperparameters

In [10]:
# Hyperparameters
epochs = 8
learning_rate = 0.01
batch_size = 32
train_samples = len(train_data)
batches = train_samples // batch_size

In [9]:
params = np.random.normal(loc=0.0, scale=1.0, size = 357)

# Training

In [8]:
opt = qml.AdamOptimizer(stepsize=learning_rate)

In [ ]:
from tqdm.notebook import tqdm, trange
import traceback
history = []

for epoch in range(epochs):

    for batch, i in zip(train_loader,trange(batches)):
        
        data = batch[0]
        target = batch[1]      
                
        try:
            params, cost = opt.step_and_cost(lambda v: loss(v,data,target),
                                                      params)
        except Exception:
            print(traceback.format_exc())
               
    history.append(res)
    predicted_train = label([circuit(num_wires, num_ancilla)(x,params) for x in data])
    accuracy_train = accuracy(predicted_train , target)
    cost = loss(params, data, target)    
    res = [epoch + 1, cost, accuracy_train]
    print(
    "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f}".format(
        *res
    )
)

  0%|          | 0/773 [00:00<?, ?it/s]

Epoch:  1 | Loss: -3.327185 | Train accuracy: 0.218750


  0%|          | 0/773 [00:00<?, ?it/s]

Epoch:  2 | Loss: -2.721115 | Train accuracy: 0.125000


  0%|          | 0/773 [00:00<?, ?it/s]

...

  0%|          | 0/773 [00:00<?, ?it/s]

Epoch:  5 | Loss: -3.311690 | Train accuracy: 0.093750


  0%|          | 0/773 [00:00<?, ?it/s]

Epoch:  6 | Loss: -3.517539 | Train accuracy: 0.218750


  0%|          | 0/773 [00:00<?, ?it/s]

# Testing

In [1]:
predicted_test = label([circuit(x,params) for x in X_test])
accuracy_test = accuracy(predicted_test,Y_test)

NameError: name 'label' is not defined